# Computational Linear Algebra
## A great project on Singular Value Decomposition

### by Paul, Placida, and Sean

## Know your audience
- Third year students with a prior course in linear algebra
- Prior exposure to programming recommended but not required
- In person; we are going to tell ourselves that it's an active learning class
- LMS agnostic
- Access to Syzygy assumed
- Students have seen a short lecture on SVD before doing the notebook

## Big Ideas and Essential Questions
- Main idea: Singular Value Decomposition is a very stable and fast algorithm with many applications in both math and industry, and it works with **any** matrix
- Core understanding -- the decomposition provides us with lots of useful information:
     - Four fundamental subspaces
     - Rank
     - Norm of a matrix
     - Condition number: how changes in matrix entries can affect solutions
     - Pseudoinverse, least squares
     - Principal component analysis
- Basic understanding: singular values are usually not nice: this is a numerical algorithm
- Essential questions:
     - How to compute it!
     - How to use SVD to obtain all the information listed above

## Learning goals
- How to implement SVD in Python using NumPy
- Relationship between singular values and eigenvalues
- To be able to prove the SVD theorem
- What is so fundamental about the four fundamental subspaces?
- Why is SVD the right way to compute rank?
- How to apply the pseudoinverse to least squares solutions
- Geometric interpretation of singular values

## Learning plan
- Walk through the steps of the SVD algorithm in an example
- Confirm that the results match what the built-in algorithm produces
- Try the algorithm on a few more examples
- Define the four fundamental subspaces, show how to find them from SVD
- Do examples and confirm that SVD gives the same nullspace as "old" methods
- Also use SVD to determine rank in these examples
- Introduce pseudoinverse and have students work through an example

## Notebook design
- Code examples provided
- Ensure students understand how to interpret output by comparing algorithm output to initial worked example
- Students expected to modify/copy code for their own examples

# The Singular Value Decomposition

For any $m\times n$ matrix $A\text{,}$ the matrices $A^TA$ and $AA^T$ are both positive. (Exercise!) This means that we can define $\sqrt{A^TA}\text{,}$ even if $A$ itself is not symmetric or positive.

- Since $A^TA$ is symmetric, we know that it can be diagonalized.
- Since $A^TA$ is positive, we know its eigenvalues are non-negative.
- This means we can define the singular values $\sigma_i = \sqrt{\lambda_i}$ for each $i=1,\ldots, n$.
- This works even if $A$ is not a square matrix!

The singular value decomposition has the form

$$A = P\Sigma_A Q^T,$$

where $\Sigma_A$ is a matrix containing the singular values of $A$. There are two conventions:
1. $\Sigma_A$ has the same size as $A$, and the upper-left corner is block-diagonal, with diagonal entries given by the singular values of $A$.
2. $\Sigma_A$ is truncated to include only the diagonal matrix of singular values.

The sizes of $P$ and $Q$ depend on which convention we choose. The algorithm in `NumPy` gives the full matrices $P$ and $Q$, but does not give $\Sigma_A$; it only lists the singular values.

## 1. Initial example

Let $A = \begin{bmatrix}1&0&1\\0&1&2\end{bmatrix}$. Compute $A^TA$, and find the singular values of $A$ by determining the eigenvalues of $A^TA$.

First, let's load the required libraries.

In [1]:
import numpy as np
import scipy.linalg as la

Next, let's define our matrix $A$ as a NumPy array, and compute $B=A^TA$.

In [2]:
A = np.array([[1,0,1],[0,1,2]])
B = (A.T)@A
print(A)
print(B)

[[1 0 1]
 [0 1 2]]
[[1 0 1]
 [0 1 2]
 [1 2 5]]


Next, let's find the eigenvalues of $B$.

In [3]:
eigvals = la.eig(B)[0].real
print(eigvals)

[6.00000000e+00 1.00000000e+00 4.59534891e-17]


It looks like our eigenvalues are $6, 1$ and $0$. Now, let's get the singular values.

In [4]:
singvals = []
for ev in eigvals:
    singvals.append(np.sqrt(ev))
print(singvals)

[2.449489742783178, 1.0, 6.778900286091395e-09]


The matrix $Q$ is an orthogonal $n\times n$ matrix whose columns are an orthonormal basis of eigenvectors for $A^TA\text{.}$ The matrix $P$ is an orthogonal $m\times m$ matrix whose columns are an orthonormal basis of $\mathbb{R}^m\text{.}$ (The first $r$ columns of $P$ are given by $A\mathbf{q}_i\text{,}$ where $\mathbf{q}_i$ is the eigenvector of $A^TA$ corresponding to the positive singular value $\sigma_i\text{.}$)

First, let's get the eigenvectors.

In [5]:
eigvects = la.eig(B)[1].real
print(eigvects)

[[-1.82574186e-01 -8.94427191e-01 -4.08248290e-01]
 [-3.65148372e-01  4.47213595e-01 -8.16496581e-01]
 [-9.12870929e-01 -6.94835567e-17  4.08248290e-01]]


The columns of this matrix are the eigenvectors of $A^TA$. Since $A^TA$ is symmetric, we know that these eigenvectors are orthogonal; by default, the `la.eig` command produces unit vectors, so we can proceed directly to forming the matrix $Q$. In fact, we don't have to proceed anywhere. This **is** the matrix $Q$!

In [7]:
Q = eigvects

For later reference, we want to extract the eigenvectors, which are the columns of $Q$. The command `Q[:,i]` will extract column `i`, but it will extract it as a row vector, so we also need to reshape it as a column. We do this as follows:

In [8]:
q1 = Q[:,0].reshape(3,1)
q2 = Q[:,1].reshape(3,1)
q3 = Q[:,2].reshape(3,1)

Next, we want to construct the matrix $P$. The columns of $P$ are eigenvectors of $AA^T$, so we proceed as above.

In [9]:
C = A@(A.T)
Ceval,Cevec = la.eig(C)

In [10]:
P = Cevec
print(P)

[[-0.89442719 -0.4472136 ]
 [ 0.4472136  -0.89442719]]


Note: we want the columns in order of decreasing eigenvalue, so we had to swap the order of the eigenvectors above.

We want to see if this works! We're going to check two things:
1. Whether or not $P\Sigma_AQ^T$ is equal to $A$
2. If the results we found here agree with the built in `np.svd` command.

First, we need to construct $\Sigma_A$. Let's just do that by hand.

In [11]:
sigA = np.array([[singvals[0],0,0],[0,singvals[1],0]])
print(sigA)

[[2.44948974 0.         0.        ]
 [0.         1.         0.        ]]


In [12]:
P@sigA@Q.T

array([[ 0.8,  0.6,  2. ],
       [ 0.6, -0.8, -1. ]])

It didn't work! Two things could have gone wrong. First, we need to ensure that the order of our eigenvectors for both $P$ and $Q$ is consistent with the decreasing order of singular values. Let's check how the eigenvalues are ordered for $P$.

In [13]:
print(Ceval.real)

[1. 6.]


Oh, dang! Our vectors were in the wrong order! Let's look at the eigenvectors again.

In [14]:
print(Cevec.real)

[[-0.89442719 -0.4472136 ]
 [ 0.4472136  -0.89442719]]


Can we swap the columns without manually writing out the entries? Swapping rows is easy, so let's try this: transpose, to turn columns into rows, then, swap rows, and then, transpose back.

In [15]:
Cevec2 = Cevec.T
Cevec3 = np.array([Cevec2[1],Cevec2[0]])
Cevec4 = Cevec3.T
print(Cevec4)

[[-0.4472136  -0.89442719]
 [-0.89442719  0.4472136 ]]


Success! Now, let's see if we get back the matrix $A$.

In [16]:
P = Cevec4
print(P@sigA@Q.T)
print(np.round(P@sigA@Q.T))

[[ 1.00000000e+00  9.25669109e-17  1.00000000e+00]
 [-1.79128377e-16  1.00000000e+00  2.00000000e+00]]
[[ 1.  0.  1.]
 [-0.  1.  2.]]


Hooray!!!! It worked! But there is one other thing that could go wrong: each eigenvector is determined only up to sign. Change the signs on one column, and you might not get back your matrix. How can we make sure everything matches up?

### Alternative construction of $P$

Following the text by Keith Nicholson, we get the columns of $P$ using the formula
$$p_i = \frac{1}{\lVert Aq_i\rVert}q_i,$$
where $p_i,q_i$ represent the $i$th columns of $P$ and $Q$, respectively.

**Caution**: it could happen that the rank of $A$ is less than the size of $P$. (For example, if $A$ is $m\times n$ with $m>n$.) When this happens, the matrix $P$ has more columns than the matrix $Q$.
The formula above will give us a basis for a subspace, which we then need to extend to a basis of $\mathbb{R}^m$, and then orthonormalize using Gram-Schmidt. (So you might prefer to just find the eigenvalues of $AA^T$!)

Earlier, we turned the eigenvectors of $A^TA$ into column vectors called `q1,q2,q3`. The first two of these correspond to the non-zero singular values. Let's multiply them by $A$, and then normalize.

In [17]:
p1 = A@q1
p2 = A@q2
print(p1)
print(p2)

[[-1.09544512]
 [-2.19089023]]
[[-0.89442719]
 [ 0.4472136 ]]


In [18]:
p1n = (1/np.linalg.norm(p1))*p1
p2n = (1/np.linalg.norm(p2))*p2
print(p1n)
print(p2n)

[[-0.4472136 ]
 [-0.89442719]]
[[-0.89442719]
 [ 0.4472136 ]]


We could reshape these into rows, put them into an `np.array`, and then take the transpose.
Or, we could take advantage of the `hstack` function:

In [19]:
P = np.hstack((p1n,p2n))
print(P)

[[-0.4472136  -0.89442719]
 [-0.89442719  0.4472136 ]]


That's the same matrix $P$ as before!

Finally, we should note that there is a built-in function to do this for us in `Numpy`:

In [21]:
P,S,Q = np.linalg.svd(A)
print(P)
print(S)
print(Q)

[[-0.4472136  -0.89442719]
 [-0.89442719  0.4472136 ]]
[2.44948974 1.        ]
[[-1.82574186e-01 -3.65148372e-01 -9.12870929e-01]
 [-8.94427191e-01  4.47213595e-01  2.62514530e-16]
 [-4.08248290e-01 -8.16496581e-01  4.08248290e-01]]


That was much less work! (But it's good to know where these things come from.)

Note that `S` is just the list of singular values; it is not the matrix $\Sigma_A$. If you need this matrix, you will still need to construct it.

## 2. Try it yourself!

Using `np.linalg.svd`, find the singular value decomposition of the matrix
$$A = \begin{bmatrix}2&-2\\1&-1\\-1&1\end{bmatrix}.$$

Your matrices should be given the following names:
- `A` for the original matrix
- `P`, `S`, `Q` for the output of `np.linalg.svd`
- `sigA` for the matrix of singular values

In [22]:
### BEGIN SOLUTION
A = np.array([[2,-2],[1,-1],[-1,1]])
### END SOLUTION

In [23]:
"Enter the matrix A (1 mark)"
assert isinstance(A,np.ndarray), "A should be a NumPy array"
assert A.shape == (3,2), "A should be a 3 by 2 matrix"
print("You entered a matrix of the correct size!")

'Enter the matrix A'

In [25]:
### BEGIN SOLUTION
P,S,Q = np.linalg.svd(A)
### END SOLUTION

In [27]:
"Compute the SVD using NumPy (1 mark)"
assert isinstance(P,np.ndarray), "P should be a NumPy array"
assert P.shape == (3,3), "P should be 3 by 3"
assert isinstance(Q,np.ndarray), "Q should be a NumPy array"
assert Q.shape == (2,2), "Q should be 2 by 2"
print("Success!")

Success!


In [28]:
### BEGIN SOLUTION
print(S)
### END SOLUTION

[3.46410162e+00 3.72077957e-17]


In [30]:
### BEGIN SOLUTION
sigA = np.array([[S[0],0],[0,0],[0,0]])
### END SOLUTION

In [32]:
"Construct the matrix sigA (1 mark)"
assert isinstance(sigA,np.ndarray), "sigA should be a NumPy array"
assert sigA.shape == (3,2), "sigA should be 3 by 2"
assert sigA[0,0] == S[0], "first entry should be the nonzero singular value"
print("Success")

In [33]:
### BEGIN SOLUTION
A2 = np.round(P@sigA@Q.T)
print(A2)
### END SOLUTION

[[ 2. -2.]
 [ 1. -1.]
 [-1.  1.]]


In [36]:
"Check that we get back the original matrix (1 mark)"
assert np.round(P@sigA@Q.T).all() == A.all(), "these should match"
print("You did it!")

You did it!


## 3. Rank, and fundamental subspaces

In your first course in linear algebra, the rank of a matrix $A$ was defined as the number of leading ones in the reduced row-echelon form of $A$.

With SVD, we can give a more robust defintion of rank:

**Definition**: the *rank* of a matrix $A$ is the number of nonzero singular values of $A$.

There are also four "fundamental" subspaces associated with any matrix $A$:

1. The nullspace of $A$
2. The column space of $A$
3. The row space of $A$
4. The nullspace of $A^T$

These can be obtained from the SVD as follows:

Suppose the rank of $A$ is $r$. Then:
- The column space of $A$ is spanned by the first $r$ columns of $P$
- The remaining columns of $P$ give an orthonormal basis for the nullspace of $A^T$
- The first $r$ columns of $Q$ give an orthonormal basis for the row space of $A$
- The remaining columns of $Q$ give an orthonormal basis for the nullspace of $A$

### Part (a): determine the rank of the matrix $A$ from Question 2

Since NumPy uses floating point calculations, we need to treat any sufficiently small number as zero.

Below is a short program to count the number of nonzero singular values. Use this to determine the rank of $A$, and then enter the value of the rank. Use `r` as the name of the rank.

In [41]:
r = 0
for s in S:
    if np.abs(s) > 1e-5:
        r = r+1
    else:
        r = r
print(r)

1


In [42]:
### BEGIN SOLUTION
r = 1
### END SOLUTION

In [43]:
"Check that the rank is correct"
assert r == 1, "The rank should be 1"
print("Hoorary!")

Hoorary!


### Part (b): Determine the column space of $A$

Enter the vectors that form a basis for the column space of $A$. Name these `p1`, `p2`, etc.

You can leave each vector as a NumPy `array` (you don't need to reshape as a column vector).

In [45]:
### BEGIN SOLUTION
p1 = P[:,0]
print(p1)
### END SOLUTION

[-0.81649658 -0.40824829  0.40824829]


In [47]:
"Check the column space basis"
assert isinstance(p1,np.ndarray), "p1 should be a NumPy array"
assert p1.all() == P[:,0].all(), "p1 should be the first column of P"
print("correct!")

correct!


### Part (c) Determine the nullspace of $A$

Enter the vectors that form a basis for the nullspace of $A$. Name these `q1, q2`, etc.

You can leave each vector as a NumPy `array` (you don't need to reshape as a column vector).

In [49]:
### BEGIN SOLUTION
q1 = Q[:,1]
### END SOLUTION

In [50]:
"Check the nullspace basis"
assert isinstance(q1,np.ndarray), "q1 should be a NumPy array"
assert q1.all() == Q[:,1].all(), "q1 should be the first column of Q"
print("correct!")

correct!
